In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import EarlyStopping

from catboost import CatBoostRegressor

from math import sqrt 
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
print ('Finished imports...')

Finished imports...


In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
train.shape, test.shape

((30557, 82), (16136, 77))

In [4]:
train.head(2)

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.0,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.00004,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.6,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,33.089495,0.000114,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.00005,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652


In [5]:
test.head(2)

,Place_ID X Date,Date,Place_ID,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,0OS9LVX X 2020-01-02,2020-01-02,0OS9LVX,11.600000,30.200001,0.00409,14.656824,3.956377,0.712605,0.000053,0.000108,0.466171,0.010752,835670.492740,68.099367,1.445658,-95.984984,22.942019,0.000046,6156.074219,0.000007,0.113310,227.467539,0.032071,68.099367,1.445658,-95.984984,22.942019,0.017953,841.142869,155.982981,835625.785337,32.709708,1.898112,-95.987015,22.950150,0.000073,0.032071,68.099367,1.445658,-95.984984,22.942019,0.000088,1.494039,2592.634167,74028.429228,0.032268,6.317523,2593.137433,74023.930722,68.099367,1.445658,-95.984984,22.942019,0.299053,0.466173,835670.492740,68.099367,1.445658,-95.984984,22.942019,0.000221,0.784436,0.000184,-0.140458,0.032071,68.099367,1.445658,-95.984984,22.942019,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0OS9LVX X 2020-01-03,2020-01-03,0OS9LVX,18.300001,42.900002,0.00595,15.026544,4.230430,0.661892,0.000050,0.000109,-0.213659,0.028307,835281.882757,75.936844,34.638933,-95.017976,18.539524,0.000045,7311.869141,0.000005,0.110397,227.788713,0.044784,75.936845,34.639165,-95.017596,18.539491,0.019576,1187.570320,922.385833,835225.278332,73.808143,34.720869,-95.017095,18.548551,0.000048,0.040803,75.936813,34.641758,-95.014908,18.539116,0.000074,1.534485,7334.601102,48466.653300,0.046211,8.308433,7902.937546,46345.613031,75.936845,34.639165,-95.017596,18.539491,0.290397,-0.213657,835281.882757,75.936844,34.638933,-95.017976,18.539524,0.000034,0.678988,0.000014,-0.842713,0.040803,75.936813,34.641758,-95.014908,18.539116,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
train['Date'] = pd.to_datetime(train['Date'])
train['Month'] = train['Date'].dt.month
train['Day'] = train['Date'].dt.day
train['Dayofweek'] = train['Date'].dt.dayofweek
train['DayOfyear'] = train['Date'].dt.dayofyear
train['DayOfyear'] = train['Date'].dt.dayofyear

test['Date'] = pd.to_datetime(test['Date'])
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['Dayofweek'] = test['Date'].dt.dayofweek
test['DayOfyear'] = test['Date'].dt.dayofyear
test['DayOfyear'] = test['Date'].dt.dayofyear

In [7]:
train.head(2)

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,Month,Day,Dayofweek,DayOfyear
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.0,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.00004,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,1,2,3,2
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.6,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.022017,1148.985447,61.216687,841116.763051,-57.015200,61.402626,-74.457583,33.089495,0.000114,0.059433,-14.708036,59.624912,-67.693509,28.614804,0.000171,0.858446,175.019862,99354.162958,0.059358,5.958538,175.072145,99353.672374,-14.708036,59.624912,-67.693509,28.614804,0.315403,-1.082553,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000150,0.433957,0.00005,-1.452612,0.059433,-14.708036,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652,1,3,4,3


In [8]:
train.drop(['Place_ID X Date','Date','Place_ID','target_min','target_max','target_variance','target_count'], axis=1, inplace=True)
test.drop(['Place_ID X Date','Date','Place_ID'], axis=1, inplace=True)

In [9]:
train.shape, test.shape

((30557, 79), (16136, 78))

In [10]:
lag_features = ['relative_humidity_2m_above_ground',
                'v_component_of_wind_10m_above_ground',
                'L3_CLOUD_surface_albedo',
                'L3_CO_CO_column_number_density',
                'temperature_2m_above_ground',
                'L3_CLOUD_cloud_optical_depth',
                'L3_O3_O3_column_number_density'
                ]

for feat in lag_features:
    for i in range(3):
        train[feat+'_lag'+str(i+1)] = train[feat].shift(i+1)
        test[feat+'_lag'+str(i+1)] = test[feat].shift(i+1)

In [11]:
X = train.drop(labels=['target'], axis=1)
y = train['target'].values

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=1)

In [13]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((22917, 99), (22917,), (7640, 99), (7640,))

In [14]:
X.head(5)

,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,specific_humidity_2m_above_ground,temperature_2m_above_ground,u_component_of_wind_10m_above_ground,v_component_of_wind_10m_above_ground,L3_NO2_NO2_column_number_density,L3_NO2_NO2_slant_column_number_density,L3_NO2_absorbing_aerosol_index,L3_NO2_cloud_fraction,L3_NO2_sensor_altitude,L3_NO2_sensor_azimuth_angle,L3_NO2_sensor_zenith_angle,L3_NO2_solar_azimuth_angle,L3_NO2_solar_zenith_angle,L3_NO2_stratospheric_NO2_column_number_density,L3_NO2_tropopause_pressure,L3_NO2_tropospheric_NO2_column_number_density,L3_O3_O3_column_number_density,L3_O3_O3_effective_temperature,L3_O3_cloud_fraction,L3_O3_sensor_azimuth_angle,L3_O3_sensor_zenith_angle,L3_O3_solar_azimuth_angle,L3_O3_solar_zenith_angle,L3_CO_CO_column_number_density,L3_CO_H2O_column_number_density,L3_CO_cloud_height,L3_CO_sensor_altitude,L3_CO_sensor_azimuth_angle,L3_CO_sensor_zenith_angle,L3_CO_solar_azimuth_angle,L3_CO_solar_zenith_angle,L3_HCHO_HCHO_slant_column_number_density,L3_HCHO_cloud_fraction,L3_HCHO_sensor_azimuth_angle,L3_HCHO_sensor_zenith_angle,L3_HCHO_solar_azimuth_angle,L3_HCHO_solar_zenith_angle,L3_HCHO_tropospheric_HCHO_column_number_density,L3_HCHO_tropospheric_HCHO_column_number_density_amf,L3_CLOUD_cloud_base_height,L3_CLOUD_cloud_base_pressure,L3_CLOUD_cloud_fraction,L3_CLOUD_cloud_optical_depth,L3_CLOUD_cloud_top_height,L3_CLOUD_cloud_top_pressure,L3_CLOUD_sensor_azimuth_angle,L3_CLOUD_sensor_zenith_angle,L3_CLOUD_solar_azimuth_angle,L3_CLOUD_solar_zenith_angle,L3_CLOUD_surface_albedo,L3_AER_AI_absorbing_aerosol_index,L3_AER_AI_sensor_altitude,L3_AER_AI_sensor_azimuth_angle,L3_AER_AI_sensor_zenith_angle,L3_AER_AI_solar_azimuth_angle,L3_AER_AI_solar_zenith_angle,L3_SO2_SO2_column_number_density,L3_SO2_SO2_column_number_density_amf,L3_SO2_SO2_slant_column_number_density,L3_SO2_absorbing_aerosol_index,L3_SO2_cloud_fraction,L3_SO2_sensor_azimuth_angle,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,Month,Day,Dayofweek,DayOfyear,relative_humidity_2m_above_ground_lag1,relative_humidity_2m_above_ground_lag2,relative_humidity_2m_above_ground_lag3,v_component_of_wind_10m_above_ground_lag1,v_component_of_wind_10m_above_ground_lag2,v_component_of_wind_10m_above_ground_lag3,L3_CLOUD_surface_albedo_lag1,L3_CLOUD_surface_albedo_lag2,L3_CLOUD_surface_albedo_lag3,L3_CO_CO_column_number_density_lag1,L3_CO_CO_column_number_density_lag2,L3_CO_CO_column_number_density_lag3,temperature_2m_above_ground_lag1,temperature_2m_above_ground_lag2,temperature_2m_above_ground_lag3,L3_CLOUD_cloud_optical_depth_lag1,L3_CLOUD_cloud_optical_depth_lag2,L3_CLOUD_cloud_optical_depth_lag3,L3_O3_O3_column_number_density_lag1,L3_O3_O3_column_number_density_lag2,L3_O3_O3_column_number_density_lag3
0,11.000000,60.200001,0.00804,18.516840,1.996377,-1.227395,0.000074,0.000156,-1.231330,0.006507,840209.874619,76.537512,38.634284,-61.736719,22.358167,0.000057,6156.074219,0.000017,0.119095,234.151102,0.000000,76.536426,38.593017,-61.752587,22.363665,0.021080,883.332451,267.017184,840138.461052,74.543393,38.622451,-61.789016,22.379054,-0.000010,0.000000,76.536426,38.593017,-61.752587,22.363665,0.000064,0.566828,NaN,NaN,0.000000,NaN,NaN,NaN,76.536426,38.593017,-61.752587,22.363665,NaN,-1.231330,840209.874619,76.537512,38.634284,-61.736719,22.358167,-0.000127,0.312521,-0.000040,-1.861476,0.000000,76.536426,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,1,2,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.600000,48.799999,0.00839,22.546533,3.330430,-1.188108,0.000076,0.000197,-1.082553,0.018360,840772.941995,-14.708036,59.624912,-67.693509,28.614804,0.000055,6156.074219,0.000021,0.115179,233.313706

Model

In [42]:
errlgb = []
y_pred_totlgb = []
Xtest = test

fold = KFold(n_splits=5, shuffle=True, random_state=2)

for train_index, test_index in fold.split(X):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    cb_model = CatBoostRegressor(iterations=8000,
                                learning_rate=0.05,
                                depth=8,
                                eval_metric='RMSE',
                                random_seed = 42,
                                bagging_temperature = 0.2,
                                od_type='Iter',
                                metric_period = 50,
                                od_wait=300) 
    cb_model.fit(X, y,
                 use_best_model=True,
                 verbose=50)

    y_pred = cb_model.predict(X_test) 

    print("RMSE: ", np.sqrt(mean_squared_error(y_test, y_pred)))
    
    errlgb.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    p = cb_model.predict(Xtest)
    
    y_pred_totlgb.append(p)

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 46.0946619	total: 103ms	remaining: 13m 41s
50:	learn: 33.2389267	total: 5.11s	remaining: 13m 16s
100:	learn: 30.4596398	total: 9.38s	remaining: 12m 13s
150:	learn: 28.8988376	total: 13.2s	remaining: 11m 25s
200:	learn: 27.6282561	total: 17.2s	remaining: 11m 8s
250:	learn: 26.4672692	total: 21.7s	remaining: 11m 10s
300:	learn: 25.3463108	total: 25.7s	remaining: 10m 56s
350:	learn: 24.3619214	total: 29.6s	remaining: 10m 45s
400:	learn: 23.5057440	total: 33.6s	remaining: 10m 35s
450:	learn: 22.7756116	total: 37.5s	remaining: 10m 27s
500:	learn: 22.1114179	total: 41.3s	remaining: 10m 18s
550:	learn: 21.4596370	total: 45.2s	remaining: 10m 10s
600:	learn: 20.8513551	total: 49.1s	remaining: 10m 4s
650:	learn: 20.2557189	total: 53.4s	remaining: 10m 2s
700:	learn: 19.7728836	total: 58.2s	remaining: 10m 6s
750:	learn: 19.2616131	total: 1m 2s	remaining: 10m 3s
800:	learn: 18.7671754	total: 1m 7s	remaining: 10m 8s
850:	learn: 18.3576808	total: 1m 13s	remaining: 10m 14s
900:	learn: 17.921

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 46.0946619	total: 143ms	remaining: 19m 2s
50:	learn: 33.2389267	total: 4.8s	remaining: 12m 27s
100:	learn: 30.4596398	total: 8.77s	remaining: 11m 26s
150:	learn: 28.8988376	total: 12.7s	remaining: 10m 58s
200:	learn: 27.6282561	total: 16.6s	remaining: 10m 45s
250:	learn: 26.4672692	total: 20.8s	remaining: 10m 43s
300:	learn: 25.3463108	total: 25s	remaining: 10m 40s
350:	learn: 24.3619214	total: 29.5s	remaining: 10m 43s
400:	learn: 23.5057440	total: 34.7s	remaining: 10m 57s
450:	learn: 22.7756116	total: 38.8s	remaining: 10m 49s
500:	learn: 22.1114179	total: 43.1s	remaining: 10m 45s
550:	learn: 21.4596370	total: 47.5s	remaining: 10m 42s
600:	learn: 20.8513551	total: 51.3s	remaining: 10m 31s
650:	learn: 20.2557189	total: 55.8s	remaining: 10m 30s
700:	learn: 19.7728836	total: 1m	remaining: 10m 31s
750:	learn: 19.2616131	total: 1m 4s	remaining: 10m 26s
800:	learn: 18.7671754	total: 1m 8s	remaining: 10m 19s
850:	learn: 18.3576808	total: 1m 14s	remaining: 10m 23s
900:	learn: 17.9217

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 46.0946619	total: 92.5ms	remaining: 12m 20s
50:	learn: 33.2389267	total: 4.13s	remaining: 10m 44s
100:	learn: 30.4596398	total: 7.85s	remaining: 10m 13s
150:	learn: 28.8988376	total: 11.6s	remaining: 10m
200:	learn: 27.6282561	total: 15.4s	remaining: 9m 57s
250:	learn: 26.4672692	total: 19.1s	remaining: 9m 50s
300:	learn: 25.3463108	total: 23s	remaining: 9m 47s
350:	learn: 24.3619214	total: 26.7s	remaining: 9m 42s
400:	learn: 23.5057440	total: 30.5s	remaining: 9m 37s
450:	learn: 22.7756116	total: 34.2s	remaining: 9m 31s
500:	learn: 22.1114179	total: 37.9s	remaining: 9m 27s
550:	learn: 21.4596370	total: 41.7s	remaining: 9m 23s
600:	learn: 20.8513551	total: 45.4s	remaining: 9m 19s
650:	learn: 20.2557189	total: 49.2s	remaining: 9m 15s
700:	learn: 19.7728836	total: 52.9s	remaining: 9m 10s
750:	learn: 19.2616131	total: 56.6s	remaining: 9m 6s
800:	learn: 18.7671754	total: 1m	remaining: 9m 3s
850:	learn: 18.3576808	total: 1m 4s	remaining: 9m 1s
900:	learn: 17.9217011	total: 1m 8s	re

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 46.0946619	total: 93.8ms	remaining: 12m 29s
50:	learn: 33.2389267	total: 4.02s	remaining: 10m 26s
100:	learn: 30.4596398	total: 7.79s	remaining: 10m 9s
150:	learn: 28.8988376	total: 11.6s	remaining: 10m 3s
200:	learn: 27.6282561	total: 15.3s	remaining: 9m 55s
250:	learn: 26.4672692	total: 19s	remaining: 9m 47s
300:	learn: 25.3463108	total: 22.7s	remaining: 9m 41s
350:	learn: 24.3619214	total: 26.5s	remaining: 9m 36s
400:	learn: 23.5057440	total: 30.4s	remaining: 9m 35s
450:	learn: 22.7756116	total: 34.2s	remaining: 9m 32s
500:	learn: 22.1114179	total: 37.9s	remaining: 9m 27s
550:	learn: 21.4596370	total: 41.8s	remaining: 9m 25s
600:	learn: 20.8513551	total: 45.5s	remaining: 9m 20s
650:	learn: 20.2557189	total: 49.4s	remaining: 9m 17s
700:	learn: 19.7728836	total: 53.1s	remaining: 9m 13s
750:	learn: 19.2616131	total: 56.9s	remaining: 9m 9s
800:	learn: 18.7671754	total: 1m	remaining: 9m 5s
850:	learn: 18.3576808	total: 1m 4s	remaining: 9m 1s
900:	learn: 17.9217011	total: 1m 8s	

You should provide test set for use best model. use_best_model parameter has been switched to false value.


0:	learn: 46.0946619	total: 96.8ms	remaining: 12m 53s
50:	learn: 33.2389267	total: 4.06s	remaining: 10m 32s
100:	learn: 30.4596398	total: 7.89s	remaining: 10m 17s
150:	learn: 28.8988376	total: 11.8s	remaining: 10m 13s
200:	learn: 27.6282561	total: 15.7s	remaining: 10m 7s
250:	learn: 26.4672692	total: 19.5s	remaining: 10m 2s
300:	learn: 25.3463108	total: 23.5s	remaining: 10m 1s
350:	learn: 24.3619214	total: 27.4s	remaining: 9m 56s
400:	learn: 23.5057440	total: 31.3s	remaining: 9m 53s
450:	learn: 22.7756116	total: 35.2s	remaining: 9m 49s
500:	learn: 22.1114179	total: 39.7s	remaining: 9m 54s
550:	learn: 21.4596370	total: 44s	remaining: 9m 54s
600:	learn: 20.8513551	total: 48.1s	remaining: 9m 52s
650:	learn: 20.2557189	total: 53.2s	remaining: 10m
700:	learn: 19.7728836	total: 57.1s	remaining: 9m 54s
750:	learn: 19.2616131	total: 1m 1s	remaining: 9m 49s
800:	learn: 18.7671754	total: 1m 4s	remaining: 9m 43s
850:	learn: 18.3576808	total: 1m 8s	remaining: 9m 38s
900:	learn: 17.9217011	total: 1

In [44]:
np.mean(errlgb, 0)

2.3185624317157894

In [45]:
y_pred = np.mean(y_pred_totlgb,0)

In [46]:
y_pred

array([33.71760034, 31.99859936, 31.7223561 , ..., 45.64300881,
       46.27785434, 34.64504955])

In [47]:
sub= pd.read_csv('SampleSubmission (1).csv')

In [48]:
sub['target'] = y_pred

In [49]:
sub.head()

,Place_ID X Date,target
0,0OS9LVX X 2020-01-02,33.717600
1,0OS9LVX X 2020-01-03,31.998599
2,0OS9LVX X 2020-01-04,31.722356
3,0OS9LVX X 2020-01-05,36.310960
4,0OS9LVX X 2020-01-06,30.220194


In [50]:
sub.to_csv('submission_ze1.csv', index=False)

In [54]:
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
import numpy as np

err_gbr = []
y_pred_tot_gbr = []
X_test = test  # Assuming 'test' is your test dataset

# Instantiate the imputer object
imputer = SimpleImputer(strategy='mean')  # You can specify other strategies as well


fold = KFold(n_splits=6, shuffle=True, random_state=2)

for train_index, test_index in fold.split(X):
    
    X_train, X_val = X.loc[train_index], X.loc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    
    # Impute missing values in training and validation data
    X_train_imputed = imputer.fit_transform(X_train)
    X_val_imputed = imputer.transform(X_val)

    params = {
        # 'n_estimators': 1000,
        'max_iter': 1000,
        'learning_rate': 0.05,
        'max_depth': 8,
        # 'subsample': 0.8,
        'l2_regularization':0.0,
        'random_state': 42
    }

    # model = GradientBoostingRegressor(**params)
    model = HistGradientBoostingClassifier(**params)

    # model.fit(X_train, y_train)
    model.fit(X_train_imputed, y_train)

    # y_pred = model.predict(X_val)
    y_pred = model.predict(X_val_imputed)
    

    print("RMSE: ", np.sqrt(mean_squared_error(y_val, y_pred)))
    
    err_gbr.append(np.sqrt(mean_squared_error(y_val, y_pred)))
    
    p = model.predict(X_test)
    
    y_pred_tot_gbr.append(p)

# Calculate the mean RMSE across all folds
mean_rmse = np.mean(err_gbr)
print("Mean RMSE:", mean_rmse)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
Y_pred = np.mean(y_pred_tot_gbr,0)

In [ ]:
sub= pd.read_csv('SampleSubmission.csv')

In [ ]:
sub['target'] = Y_pred

In [ ]:
sub.to_csv('submission_ze2.csv', index=False)

In [25]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np

err_svm = []
y_pred_tot_svm = []
X_test = test  # Assuming 'test' is your test dataset

fold = KFold(n_splits=10, shuffle=True, random_state=2)

for train_index, test_index in fold.split(X):
    
    X_train, X_val = X.loc[train_index], X.loc[test_index]
    y_train, y_val = y[train_index], y[test_index]

    params = {
        'kernel': 'rbf',  # Radial Basis Function kernel
        'C': 1.0,  # Regularization parameter
        'epsilon': 0.1,  # Epsilon in the epsilon-SVR model
    }

    model = SVR(**params)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    print("RMSE: ", np.sqrt(mean_squared_error(y_val, y_pred)))
    
    err_svm.append(np.sqrt(mean_squared_error(y_val, y_pred)))
    
    p = model.predict(X_test)
    
    y_pred_tot_svm.append(p)

# Calculate the mean RMSE across all folds
mean_rmse = np.mean(err_svm)
print("Mean RMSE:", mean_rmse)

ValueError: Input X contains NaN.
SVR does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
Y_pred = np.mean(y_pred_tot_svm,0)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(cb_model.get_feature_importance(), X.columns), reverse=True)[:], columns=['Value','Feature'])
plt.figure(figsize=(20,20))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('Catboost Features')
plt.tight_layout()
plt.show()

Prediction

In [ ]:
Xtest = test

In [ ]:
errlgb = []
y_pred_totlgb = []

fold = KFold(n_splits=10, shuffle=True, random_state=2)

for train_index, test_index in fold.split(X):
    
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    cb_model = CatBoostRegressor(iterations=5000,
                                 learning_rate=0.045,
                                 depth=8,
                                 eval_metric='RMSE',
                                 random_seed = 42,
                                 bagging_temperature = 0.2,
                                 od_type='Iter',
                                 metric_period = 50,
                                 od_wait=300)
    cb_model.fit(X, y,
                 use_best_model=True,
                 verbose=50)

    y_pred = cb_model.predict(X_test) 

    print("RMSE: ", np.sqrt(mean_squared_error(y_test, y_pred)))
    
    errlgb.append(np.sqrt(mean_squared_error(y_test, y_pred)))
    
    p = cb_model.predict(Xtest)
    
    y_pred_totlgb.append(p)

In [ ]:
np.mean(y_pred_totlgb,0)

In [ ]:
np.mean(errlgb, 0)

Submission

In [ ]:
y_pred = np.mean(y_pred_totlgb,0)

In [ ]:
sub= pd.read_csv('SampleSubmission.csv')

In [ ]:
sub['target'] = y_pred

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission_urban2.csv', index=False)